# Part II: Statistical Analysis for Reproducing Paper Figures

This notebook contains the statistical analysis code used in the paper:  
**"Insights from Reinforcement Learning and individual-based model simulations on population compliance and policy optimization during COVID-19"**

The notebook analyzes output data from agent-based simulations optimized with reinforcement learning (SiRL).  
The goal is to evaluate how different vaccination order strategies and compliance levels affect public health and economic outcomes.

## What this notebook includes:
- Interaction effects on the cumulative economic index (Figure 5)


---

## Dataset Location and Loading Instructions

This analysis is based on the simulation output file:

experiment2_results_UP.csv

If you are running this notebook on **Google Colab**, follow these steps to load the dataset from your Google Drive:

1. Mount your Google Drive:

```python
from google.colab import drive
drive.mount('/content/drive')

	2.	Load the dataset:

import pandas as pd

# === Dataset Loading ===
data = pd.read_csv("/content/drive/Name_Your_Drive/path_to_the_data_file", header=0)

Make sure to replace "Name_Your_Drive/path_to_the_data_file" with the full path to experiment1_results_UP.csv inside your own Google Drive.

Note:
This notebook includes sections titled
## Supplementary Visualizations (Not Included in the Published Article)
These parts contain extended plots and visual summaries that go beyond what was included in the published article.
They are intended to help better understand the simulation dynamics and provide additional context for interpreting the results.


In [ ]:
# === Import Required Libraries ===
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, f_oneway, sem, t
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive, files
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

# === Step 1: Mount Google Drive ===
# This step enables access to files stored in your personal Google Drive.
drive.mount('/content/drive')

# === Step 2: Load Simulation Dataset ===
# Read the CSV file containing simulation results.
# Note: 'header=1' assumes that the actual column names start from the second row.
data = pd.read_csv("/content/drive/Name_Your_Drive/path_to_the_data_file", header=0)

# === Step 3: Clean Column Names ===
# Remove any leading/trailing whitespace from column headers for consistency.
data.columns = data.columns.str.strip()

# === Step 4: Create Composite Grouping Variable ===
# Combine 'City' and 'Order Vac' into a single column for grouped analyses and visualizations.
data['City_OrderVac'] = data['City'] + ' - ' + data['Order Vac']

Supplementary Visualizations

In [ ]:
# === Step 1: Preprocessing ===

# Create a new column combining City and Vaccination Order for grouped comparisons
data['City_OrderVac'] = data['City'] + ' - ' + data['Order Vac']

# === Step 2: ANOVA Model Specification ===

# Full factorial model including all main effects and interactions
formula = 'Q("Cumulative economic index") ~ C(Compliance) + C(City) + C(Q("Order Vac")) + ' \
          'C(Compliance):C(City) + C(Compliance):C(Q("Order Vac")) + C(City):C(Q("Order Vac")) + ' \
          'C(Compliance):C(City):C(Q("Order Vac"))'

# Fit the model using ordinary least squares
model = ols(formula, data=data).fit()

# Perform Type II ANOVA
anova_table = sm.stats.anova_lm(model, typ=2)

# Print the full ANOVA table
print("\n" + "="*60)
print("FULL ANOVA TABLE")
print("="*60)
print(anova_table)

# === Step 3: Filter and Rename Effects for Plotting ===

# Remove non-significant interaction effects and residuals
effects_to_remove = [
    'Residual',
    'C(Compliance):C(City)',
    'C(Compliance):C(Q("Order Vac"))'
]
filtered_anova = anova_table.drop(index=effects_to_remove)

# Rename effects for plot clarity
rename_map = {
    'C(Compliance)': 'Compliance Level',
    'C(City)': 'City (Holon vs. Bene Beraq)',
    'C(Q("Order Vac"))': 'Vaccination Order (Asc vs Desc)',
    'C(City):C(Q("Order Vac"))': 'City × Vaccination Policy',
    'C(Compliance):C(City):C(Q("Order Vac"))': 'Compliance × City × Policy'
}
filtered_anova_renamed = filtered_anova.rename(index=rename_map)

# === Step 4: Tukey HSD Tests ===

# Define groupings for multiple comparisons
groupings = {
    'Compliance': 'Compliance Levels',
    'City': 'City (Holon vs Bene Beraq)',
    'Order Vac': 'Vaccination Order (Asc vs Desc)',
    'City_OrderVac': 'City × Vaccination Policy'
}

tukey_results = {}

print("\n" + "="*60)
print("Tukey HSD RESULTS")
print("="*60)

# Run Tukey test for each grouping
for group_var, title in groupings.items():
    tukey = pairwise_tukeyhsd(endog=data['Cumulative economic index'],
                              groups=data[group_var],
                              alpha=0.05)
    tukey_results[title] = tukey
    print(f"\n--- {title} ---")
    print(tukey)

# === Step 5: Visualization ===

# Create a 2×2 grid of plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# (Top-left) Barplot of ANOVA F-values
ax = axes[0, 0]
filtered_anova_renamed["F"].plot(kind='bar', ax=ax, color='skyblue', alpha=0.85)
ax.axhline(y=4.0, color='red', linestyle='--', label='F ≈ 0.05 significance threshold')
ax.set_title("ANOVA F-Values", fontsize=14)
ax.set_ylabel("F Value")
ax.set_xticklabels(filtered_anova_renamed.index, rotation=45, ha='right')
ax.grid(True, linestyle='--', alpha=0.6)
ax.legend()

# (Other 3 plots) Tukey HSD confidence intervals
tukey_titles = list(tukey_results.items())
for i in range(3):  # Show 3 out of 4 results in this grid
    row, col = divmod(i + 1, 2)
    title, tukey = tukey_titles[i]
    ax = axes[row, col]
    tukey.plot_simultaneous(ax=ax)
    ax.set_title(f"Tukey HSD: {title}", fontsize=13)
    ax.set_xlabel("Cumulative Economic Index")
    ax.grid(True, linestyle='--', alpha=0.5)

# Layout adjustments
plt.tight_layout()
plt.show()

In [ ]:
# === Data Preprocessing ===

# Ensure the categorical variables are strings
data['Compliance'] = data['Compliance'].astype(str)
data['City'] = data['City'].astype(str)
data['Order Vac'] = data['Order Vac'].astype(str)

# Create a new feature representing the three-way interaction
data['ThreeWay'] = data['City'] + ' × ' + data['Compliance'] + ' × ' + data['Order Vac']

# === Plotting: Four Subplots Showing Factor Effects ===

# Set up the main figure with 4 vertically stacked subplots
fig, axes = plt.subplots(4, 1, figsize=(14, 29))
plt.subplots_adjust(hspace=0.6)

# (A) Effect of Compliance × City on Economic Index
sns.barplot(data=data, x='Compliance', y='Cumulative economic index',
            hue='City', ci=95, capsize=0.1, ax=axes[0],
            palette='Set2', edgecolor='black')
axes[0].set_title("(A)", fontsize=20, fontweight='bold')
axes[0].set_ylabel("Cumulative Economic Index (Mean ± 95% CI)", fontsize=15)
axes[0].tick_params(axis='x', labelsize=15)
axes[0].grid(axis='y', linestyle='--', alpha=0.5)
axes[0].text(0.03, 0.95, "F = 5.82\np = 0.0031",
             transform=axes[0].transAxes,
             ha='left', va='top',
             fontsize=18,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=0.8))

# (B) Effect of Compliance × Vaccination Order
sns.barplot(data=data, x='Compliance', y='Cumulative economic index',
            hue='Order Vac', ci=95, capsize=0.1, ax=axes[1],
            palette='Set3', edgecolor='black')
axes[1].set_title("(B)", fontsize=20, fontweight='bold')
axes[1].set_ylabel("Cumulative Economic Index (Mean ± 95% CI)", fontsize=14)
axes[1].tick_params(axis='x', labelsize=14)
axes[1].grid(axis='y', linestyle='--', alpha=0.5)
axes[1].text(0.03, 0.95, "F = 0.03\np = 0.969",
             transform=axes[1].transAxes,
             ha='left', va='top',
             fontsize=14,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=0.8))

# (C) Effect of City × Vaccination Order
sns.barplot(data=data, x='City', y='Cumulative economic index',
            hue='Order Vac', ci=95, capsize=0.1, ax=axes[2],
            palette='Set1', edgecolor='black')
axes[2].set_title("(C)", fontsize=20, fontweight='bold')
axes[2].set_ylabel("Cumulative Economic Index (Mean ± 95% CI)", fontsize=16)
axes[2].tick_params(axis='x', labelsize=16)
axes[2].grid(axis='y', linestyle='--', alpha=0.5)
axes[2].text(0.03, 0.95, "F = 190.06\np < 0.0001",
             transform=axes[2].transAxes,
             ha='left', va='top',
             fontsize=14,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=0.8))

# (D) Full Three-Way Interaction Plot
sns.barplot(data=data, x='ThreeWay', y='Cumulative economic index',
            ci=95, capsize=0.1, ax=axes[3],
            palette='Pastel2', edgecolor='black')
axes[3].set_title("(D)", fontsize=20, fontweight='bold')
axes[3].set_ylabel("Cumulative Economic Index (Mean ± 95% CI)", fontsize=14)
unique_labels = data['ThreeWay'].unique()
axes[3].set_xticks(range(len(unique_labels)))
axes[3].set_xticklabels(unique_labels, rotation=45, ha='right', fontsize=14)
axes[3].grid(axis='y', linestyle='--', alpha=0.5)
axes[3].text(0.03, 0.95, "F = 11.29\np = 0.000015",
             transform=axes[3].transAxes,
             ha='left', va='top',
             fontsize=14,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=0.8))

# === Global Title and Layout ===
plt.suptitle(
    "Estimated Marginal Means (±95% CI) from Post Hoc Analysis Following ANOVA",
    fontsize=21, fontweight='bold',
    y=0.89
)
plt.tight_layout(rect=[0, 0, 1, 0.89])

# === Save and Download ===
plt.savefig("interaction_effects_cleaned.png", dpi=300, bbox_inches='tight')
files.download("interaction_effects_cleaned.png")

plt.show()

Supplementary Visualizations

In [ ]:
# === Step 1: Create Grouping Variable ===
# Combine 'City' and 'Order Vac' to form a unique group identifier for comparison
data['City_OrderVac'] = data['City'] + ' - ' + data['Order Vac']

# === Step 2: Perform Tukey HSD Test ===
# Tukey's Honest Significant Difference test for comparing group means
mc = MultiComparison(data['Cumulative economic index'], data['City_OrderVac'])
tukey_result = mc.tukeyhsd()

# Display the summary table of pairwise comparisons
print(tukey_result.summary())

# === Step 3: Prepare Group Means and Letter Assignments ===
# Create a DataFrame with group means, sorted from highest to lowest
groups = pd.DataFrame({
    'group': mc.groupsunique,
    'mean': data.groupby('City_OrderVac')['Cumulative economic index'].mean().values
}).sort_values('mean', ascending=False).reset_index(drop=True)

# Assign Tukey grouping letters manually (for illustrative purposes)
# Note: For rigorous statistical reporting, consider using a compact letter display algorithm
groups['letter'] = list('abc') + ['d'] * (len(groups) - 3)

# === Step 4: Plot the Group Means with Assigned Letters ===
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=groups, x='group', y='mean', palette='Set2', edgecolor='black')

# Annotate each bar with its assigned Tukey letter above the bar
for i, row in groups.iterrows():
    ax.text(i, row['mean'] + 1e7, row['letter'], ha='center', va='bottom', fontsize=14, fontweight='bold')

# === Step 5: Final Plot Formatting ===
plt.title("Tukey HSD: City × Vaccination Order", fontsize=16, fontweight='bold')
plt.ylabel("Cumulative Economic Index (Mean)", fontsize=12)
plt.xlabel("Group", fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()